In [ ]:
# IMPORTAR LIBRERIAS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import seaborn as sns
from scipy import stats
from scipy.stats import spearmanr, pearsonr
import warnings
warnings.filterwarnings('ignore')
# Configuració estètica
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from scipy.stats import spearmanr, pearsonr, shapiro
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import shapiro
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
import numpy as np
# Configuración de estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")


# Si el archivo usa otro separador como punto y coma
df = pd.read_csv(
    r"C:*\ifood_df.csv",
    sep=',',  # o el delimitador que corresponda
    engine='python'
)

# Mostrar las primeras filas para verificar
print(df.head())

print("Tipos de datos por columna:")
print(df.dtypes) 
# Información detallada
print("\nInformación completa del dataset:")
print(df.info())

# Estadísticas descriptivas por tipo de dato
print("\nResumen de datos numéricos:")
print(df.describe())




✅ Librerías importadas correctamente
    Income  Kidhome  Teenhome  Recency  MntWines  MntFruits  MntMeatProducts  \
0  58138.0        0         0       58       635         88              546   
1  46344.0        1         1       38        11          1                6   
2  71613.0        0         0       26       426         49              127   
3  26646.0        1         0       26        11          4               20   
4  58293.0        1         0       94       173         43              118   

   MntFishProducts  MntSweetProducts  MntGoldProds  ...  marital_Together  \
0              172                88            88  ...                 0   
1                2                 1             6  ...                 0   
2              111                21            42  ...                 1   
3               10                 3             5  ...                 1   
4               46                27            15  ...                 0   

   marital_Widow  e

In [70]:
#cambiar tipo de la tabla income porque si hago la integración con power bi me viene con problemas

df["Income"] = df["Income"].astype(int)


In [71]:
# MÉTODO REUTILIZABLE PARA CUALQUIER GRUPO DE COLUMNAS DUMMY
def convert_dummy_to_category(df, column_prefix):
    """
    Convierte columnas dummy con un prefijo común en una columna categórica
    """
    # Encontrar todas las columnas que empiezan con el prefijo
    dummy_columns = [col for col in df.columns if col.startswith(column_prefix)]
    
    if not dummy_columns:
        print(f"No se encontraron columnas con el prefijo: {column_prefix}")
        return df
    
    print(f"Convirtiendo {len(dummy_columns)} columnas: {dummy_columns}")
    
    # Crear la nueva columna categórica
    def get_category(row):
        for col in dummy_columns:
            if row[col] == 1:
                return col.replace(f'{column_prefix}_', '')
        return 'Unknown'
    
    new_column_name = column_prefix.replace('_', '')  # Ej: 'education' en lugar de 'education_'
    df[new_column_name] = df.apply(get_category, axis=1)
    
    # Mostrar distribución
    print(f"\nDistribución de {new_column_name}:")
    print(df[new_column_name].value_counts())
    
    return df, dummy_columns

In [72]:
# Esta es la solución más robusta y eficiente para tranformar la COLUMNA EDUCATION:

education_columns = ['education_2n Cycle', 'education_Basic', 'education_Graduation', 'education_Master', 'education_PhD']

# Método más simple y confiable
def get_education_level(row):
    for col in education_columns:
        if row[col] == 1:
            return col.replace('education_', '')
    return 'Unknown'  # Para casos donde no hay ningún 1

df['Education_Level'] = df.apply(get_education_level, axis=1)

# Mostrar resultados
print("Primeras filas después de la transformación:")
print(df[education_columns + ['Education_Level', 'MntTotal']].head())

Primeras filas después de la transformación:
   education_2n Cycle  education_Basic  education_Graduation  \
0                   0                0                     1   
1                   0                0                     1   
2                   0                0                     1   
3                   0                0                     1   
4                   0                0                     0   

   education_Master  education_PhD Education_Level  MntTotal  
0                 0              0      Graduation      1529  
1                 0              0      Graduation        21  
2                 0              0      Graduation       734  
3                 0              0      Graduation        48  
4                 0              1             PhD       407  


In [73]:
# Lista de columnas de estado civil
marital_columns = ['marital_Divorced', 'marital_Married', 'marital_Single', 'marital_Together', 'marital_Widow']

# Aplicar el mismo método que usamos para educación
def get_marital_status(row):
    for col in marital_columns:
        if row[col] == 1:
            return col.replace('marital_', '')
    return 'Unknown'

df['Marital_Status'] = df.apply(get_marital_status, axis=1)

# Verificar el resultado
print("Primeras filas con estado civil:")
print(df[marital_columns + ['Marital_Status']].head(10))

Primeras filas con estado civil:
   marital_Divorced  marital_Married  marital_Single  marital_Together  \
0                 0                0               1                 0   
1                 0                0               1                 0   
2                 0                0               0                 1   
3                 0                0               0                 1   
4                 0                1               0                 0   
5                 0                0               0                 1   
6                 1                0               0                 0   
7                 0                1               0                 0   
8                 0                0               0                 1   
9                 0                0               0                 1   

   marital_Widow Marital_Status  
0              0         Single  
1              0         Single  
2              0       Together  
3              0

In [74]:
# Eliminar las columnas dummy originales
columns_to_drop = ['education_2n Cycle', 'education_Basic', 'education_Graduation', 'education_Master', 'education_PhD'] + marital_columns 

df_clean = df.drop(columns=columns_to_drop)

print(f"DataFrame original: {df.shape}")
print(f"DataFrame limpio: {df_clean.shape}")
print(f"Columnas eliminadas: {len(columns_to_drop)}")

# Mostrar las nuevas columnas creadas
print("\n🎯 NUEVAS COLUMNAS CREADAS:")
new_categorical_cols = ['Education_Level', 'Marital_Status']
for col in new_categorical_cols:
    if col in df_clean.columns:
        print(f"✅ {col}")

# Ver la estructura final del DataFrame
print(f"\n📋 COLUMNAS FINALES EN EL DATASET LIMPIO:")
print(df_clean.columns.tolist())

# Mostrar información del DataFrame limpio
print(f"\n📊 INFORMACIÓN DEL DATASET LIMPIO:")
print(df_clean.info())

# Mostrar estadísticas de las nuevas variables categóricas
print(f"\n📈 DISTRIBUCIÓN DE LAS NUEVAS VARIABLES:")
categorical_columns = ['Education_Level', 'Marital_Status', 'Campaign_Segment']
for col in categorical_columns:
    if col in df_clean.columns:
        print(f"\n{col}:")
        print(df_clean[col].value_counts())

DataFrame original: (2205, 41)
DataFrame limpio: (2205, 31)
Columnas eliminadas: 10

🎯 NUEVAS COLUMNAS CREADAS:
✅ Education_Level
✅ Marital_Status

📋 COLUMNAS FINALES EN EL DATASET LIMPIO:
['Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response', 'Age', 'Customer_Days', 'MntTotal', 'MntRegularProds', 'AcceptedCmpOverall', 'Education_Level', 'Marital_Status']

📊 INFORMACIÓN DEL DATASET LIMPIO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2205 entries, 0 to 2204
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Income               2205 non-null   int32 
 1   Kidhome             

In [75]:
print("🔍 VERIFICACIÓN COMPLETA DE DUPLICADOS")
print("="*50)

# 1. Duplicados exactos en todas las columnas
duplicados_exactos = df_clean.duplicated().sum()
print(f"1. Duplicados exactos (todas las columnas): {duplicados_exactos}")

# 2. Verificar combinaciones únicas que deberían identificar a cada cliente
print(f"\n2. Verificación de unicidad por características clave:")

# Combinaciones que deberían ser únicas para cada cliente
combinaciones_unicas = [
    ['Income', 'Age', 'Education_Level', 'Marital_Status'],
    ['Income', 'Kidhome', 'Teenhome', 'Age'],
    ['MntTotal', 'NumWebPurchases', 'NumStorePurchases', 'Age']
]

for combo in combinaciones_unicas:
    combo_existente = [col for col in combo if col in df_clean.columns]
    if combo_existente:
        valores_unicos = df_clean[combo_existente].drop_duplicates().shape[0]
        total_registros = len(df_clean)
        print(f"   - {combo_existente}: {valores_unicos} únicos de {total_registros} registros")

# 3. Análisis detallado de duplicados por columna
print(f"\n3. ANÁLISIS DETALLADO POR COLUMNA:")
columnas_analizar = ['Income', 'Age', 'Education_Level', 'Marital_Status', 'Kidhome', 'Teenhome', 'MntTotal']
for col in columnas_analizar:
    if col in df_clean.columns:
        valores_unicos = df_clean[col].nunique()
        total_registros = len(df_clean)
        porcentaje = (valores_unicos / total_registros) * 100
        print(f"   {col:<20}: {valores_unicos:>4} únicos ({porcentaje:5.1f}%)")

# 4. Buscar posibles clientes duplicados
print(f"\n4. BÚSQUEDA DE POSIBLES CLIENTES DUPLICADOS:")
caracteristicas_cliente = ['Income', 'Age', 'Education_Level', 'Marital_Status', 'Kidhome', 'Teenhome']

if all(col in df_clean.columns for col in caracteristicas_cliente):
    # Encontrar clientes con mismas características demográficas
    clientes_similares = df_clean[df_clean.duplicated(subset=caracteristicas_cliente, keep=False)]
    
    if len(clientes_similares) > 0:
        print(f"   Se encontraron {len(clientes_similares)} registros con características similares")
        print(f"   Grupos de posibles duplicados:")
        
        # Contar cuántos grupos de duplicados hay
        grupos_duplicados = clientes_similares.groupby(caracteristicas_cliente).size()
        print(f"   Número de grupos con características idénticas: {len(grupos_duplicados)}")
        
        # Mostrar algunos ejemplos
        print(f"\n   Ejemplos de posibles duplicados:")
        print(clientes_similares[caracteristicas_cliente + ['MntTotal', 'Recency']].head(6))
    else:
        print(f"   ✅ No se encontraron clientes con características demográficas idénticas")

🔍 VERIFICACIÓN COMPLETA DE DUPLICADOS
1. Duplicados exactos (todas las columnas): 184

2. Verificación de unicidad por características clave:
   - ['Income', 'Age', 'Education_Level', 'Marital_Status']: 2002 únicos de 2205 registros
   - ['Income', 'Kidhome', 'Teenhome', 'Age']: 1999 únicos de 2205 registros
   - ['MntTotal', 'NumWebPurchases', 'NumStorePurchases', 'Age']: 1973 únicos de 2205 registros

3. ANÁLISIS DETALLADO POR COLUMNA:
   Income              : 1963 únicos ( 89.0%)
   Age                 :   56 únicos (  2.5%)
   Education_Level     :    5 únicos (  0.2%)
   Marital_Status      :    5 únicos (  0.2%)
   Kidhome             :    3 únicos (  0.1%)
   Teenhome            :    3 únicos (  0.1%)
   MntTotal            :  897 únicos ( 40.7%)

4. BÚSQUEDA DE POSIBLES CLIENTES DUPLICADOS:
   Se encontraron 397 registros con características similares
   Grupos de posibles duplicados:
   Número de grupos con características idénticas: 194

   Ejemplos de posibles duplicados:
  

In [76]:
# Función para análisis profundo de duplicados
def analizar_duplicados_detallado(df):
    print("📊 ANÁLISIS DETALLADO DE DUPLICADOS")
    print("="*50)
    
    # Verificar por cada tipo de columna
    todas_columnas = df.columns.tolist()
    
    for col in todas_columnas:
        valores_unicos = df[col].nunique()
        total_registros = len(df)
        porcentaje_unicos = (valores_unicos / total_registros) * 100
        
        print(f"{col:<25}: {valores_unicos:>4} únicos ({porcentaje_unicos:5.1f}%)")
    
    # Identificar columnas con posible problema de duplicados
    print(f"\n🚨 COLUMNAS CON POSIBLES PROBLEMAS:")
    for col in todas_columnas:
        if df[col].nunique() < len(df) * 0.1:  # Menos del 10% de valores únicos
            print(f"   - {col}: {df[col].nunique()} valores únicos")

# Ejecutar análisis detallado
analizar_duplicados_detallado(df_clean)

📊 ANÁLISIS DETALLADO DE DUPLICADOS
Income                   : 1963 únicos ( 89.0%)
Kidhome                  :    3 únicos (  0.1%)
Teenhome                 :    3 únicos (  0.1%)
Recency                  :  100 únicos (  4.5%)
MntWines                 :  775 únicos ( 35.1%)
MntFruits                :  158 únicos (  7.2%)
MntMeatProducts          :  551 únicos ( 25.0%)
MntFishProducts          :  182 únicos (  8.3%)
MntSweetProducts         :  176 únicos (  8.0%)
MntGoldProds             :  212 únicos (  9.6%)
NumDealsPurchases        :   15 únicos (  0.7%)
NumWebPurchases          :   15 únicos (  0.7%)
NumCatalogPurchases      :   13 únicos (  0.6%)
NumStorePurchases        :   14 únicos (  0.6%)
NumWebVisitsMonth        :   16 únicos (  0.7%)
AcceptedCmp3             :    2 únicos (  0.1%)
AcceptedCmp4             :    2 únicos (  0.1%)
AcceptedCmp5             :    2 únicos (  0.1%)
AcceptedCmp1             :    2 únicos (  0.1%)
AcceptedCmp2             :    2 únicos (  0.1%)
Compl

In [77]:
print("🧹 PROCEDIMIENTO DE LIMPIEZA DE DUPLICADOS")
print("="*50)

# Guardar el estado original
registros_originales = len(df_clean)
print(f"Registros originales: {registros_originales}")

# 1. ELIMINAR DUPLICADOS EXACTOS (mantener el primero)
df_clean = df_clean.drop_duplicates()
duplicados_eliminados = registros_originales - len(df_clean)
print(f"Duplicados exactos eliminados: {duplicados_eliminados}")
print(f"Registros restantes: {len(df_clean)}")

🧹 PROCEDIMIENTO DE LIMPIEZA DE DUPLICADOS
Registros originales: 2205
Duplicados exactos eliminados: 184
Registros restantes: 2021


In [78]:
# 2. Análisis de duplicados por características demográficas (CORREGIDO)
caracteristicas_unicas = ['Income', 'Age', 'Education_Level', 'Marital_Status', 'Kidhome', 'Teenhome']

# Encontrar posibles duplicados restantes
duplicados_demograficos = df_clean[df_clean.duplicated(subset=caracteristicas_unicas, keep=False)]
print(f"\n🔍 CLIENTES CON MISMAS CARACTERÍSTICAS DEMOGRÁFICAS: {len(duplicados_demograficos)}")

if len(duplicados_demograficos) > 0:
    # Usar value_counts() en lugar de groupby().items()
    grupos_tamanos = duplicados_demograficos.groupby(caracteristicas_unicas).size()
    print(f"Número de grupos con características idénticas: {len(grupos_tamanos)}")
    
    # Mostrar los grupos más grandes
    print(f"\n📊 GRUPOS MÁS GRANDES (primeros 3):")
    for i, (grupo, tamano) in enumerate(grupos_tamanos.sort_values(ascending=False).head(3).items()):
        print(f"Grupo {i+1}: {tamano} clientes con características: {grupo}")
    
    # Mostrar ejemplos de estos duplicados
    print(f"\n📋 EJEMPLOS DE CLIENTES CON MISMAS CARACTERÍSTICAS:")
    ejemplos = duplicados_demograficos[caracteristicas_unicas + ['MntTotal', 'Recency', 'Response']].head(6)
    print(ejemplos)
else:
    print("✅ No hay clientes con características demográficas idénticas")


🔍 CLIENTES CON MISMAS CARACTERÍSTICAS DEMOGRÁFICAS: 38
Número de grupos con características idénticas: 19

📊 GRUPOS MÁS GRANDES (primeros 3):
Grupo 1: 2 clientes con características: (19514, 52, 'Graduation', 'Married', 1, 1)
Grupo 2: 2 clientes con características: (52278, 57, 'PhD', 'Widow', 0, 1)
Grupo 3: 2 clientes con características: (82347, 63, '2n Cycle', 'Married', 0, 0)

📋 EJEMPLOS DE CLIENTES CON MISMAS CARACTERÍSTICAS:
     Income  Age Education_Level Marital_Status  Kidhome  Teenhome  MntTotal  \
8     30351   46             PhD       Together        1         0        44   
78    81361   33      Graduation       Divorced        0         0       702   
130   65104   44          Master       Together        0         1      1003   
182   31353   43      Graduation        Married        1         1        24   
242   63342   59             PhD       Together        0         1      1080   
357   38578   48             PhD         Single        1         1        70   

   

In [79]:
# BASADO EN EL ANÁLISSO, PROCEDEMOS CON LA SIGUIENTE ESTRATEGIA:

print(f"\n🎯 DECISIÓN FINAL SOBRE DUPLICADOS:")

if len(duplicados_demograficos) == 0:
    print("✅ No hay duplicados demográficos - Proceder con creación de IDs")
else:
    print("ℹ️ Se encontraron clientes con características similares pero:")
    print("   - Pueden ser clientes diferentes con coincidencias demográficas")
    print("   - Vamos a mantener todos los registros y crear IDs únicos")
    print("   - Esto preserva la integridad de los datos de comportamiento")

# 4. CREAR COLUMNA ID ÚNICA (ahora que limpiamos duplicados exactos)
print(f"\n🆔 CREANDO COLUMNA ID ÚNICA...")

# Opción: ID secuencial con prefijo
df_clean['CustomerID'] = ['CUST_' + str(i+1).zfill(4) for i in range(len(df_clean))]

# Verificar que no hay duplicados en el ID
print(f"✅ IDs únicos creados: {df_clean['CustomerID'].nunique()}")
print(f"✅ Total de registros: {len(df_clean)}")
print(f"🔍 Verificación: {df_clean['CustomerID'].nunique() == len(df_clean)}")

# Mover la columna ID al principio
columnas_ordenadas = ['CustomerID'] + [col for col in df_clean.columns if col != 'CustomerID']
df_clean = df_clean[columnas_ordenadas]


🎯 DECISIÓN FINAL SOBRE DUPLICADOS:


ℹ️ Se encontraron clientes con características similares pero:
   - Pueden ser clientes diferentes con coincidencias demográficas
   - Vamos a mantener todos los registros y crear IDs únicos
   - Esto preserva la integridad de los datos de comportamiento

🆔 CREANDO COLUMNA ID ÚNICA...
✅ IDs únicos creados: 2021
✅ Total de registros: 2021
🔍 Verificación: True


In [80]:
# Análisis del impacto de la limpieza
print(f"\n📊 IMPACTO DE LA LIMPIEZA EN ESTADÍSTICAS CLAVE:")

# Comparar estadísticas antes y después (aproximado)
if duplicados_eliminados > 0:
    stats_columns = ['Income', 'MntTotal', 'Response', 'Age']
    print("\nComparación de estadísticas clave:")
    for col in stats_columns:
        if col in df_clean.columns:
            print(f"\n{col}:")
            print(f"  - Media: {df_clean[col].mean():.2f}")
            print(f"  - Mediana: {df_clean[col].median():.2f}")
            if col == 'Response':
                print(f"  - Tasa de conversión: {df_clean[col].mean():.2%}")

print(f"\n🎉 DATASET LIMPIO Y LISTO PARA ANÁLISIS!")
print(f"   Total de clientes únicos: {len(df_clean)}")


📊 IMPACTO DE LA LIMPIEZA EN ESTADÍSTICAS CLAVE:

Comparación de estadísticas clave:

Income:
  - Media: 51687.26
  - Mediana: 51412.00

MntTotal:
  - Media: 563.79
  - Mediana: 343.00

Response:
  - Media: 0.15
  - Mediana: 0.00
  - Tasa de conversión: 15.39%

Age:
  - Media: 51.12
  - Mediana: 50.00

🎉 DATASET LIMPIO Y LISTO PARA ANÁLISIS!
   Total de clientes únicos: 2021


In [81]:
# Reordenar columnas por bloques lógicos
new_order = [
    # 1️⃣ Identificación y demografía
    'CustomerID', 'Age', 'Income', 'Education_Level', 'Marital_Status', 'Kidhome', 'Teenhome',
    # 2️⃣ Relación con la empresa
    'Customer_Days', 'Recency', 'Complain', 'Z_CostContact', 'Z_Revenue',
    # 3️⃣ Consumo y gasto
    'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
    'MntSweetProducts', 'MntGoldProds', 'MntTotal', 'MntRegularProds',
    # 4️⃣ Canales de compra y visitas
    'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
    'NumStorePurchases', 'NumWebVisitsMonth',
    # 5️⃣ Campañas de marketing
    'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4',
    'AcceptedCmp5', 'AcceptedCmpOverall', 'Response'
]

# Aplicar el nuevo orden
df_clean = df_clean[new_order]


In [ ]:
# CORRECCIÓN DE LA RUTA - AGREGANDO NOMBRE DE ARCHIVO Y EXTENSIÓN
ruta_elegida = r"*\digital_marketing_cleaned.csv"

try:
    # Asegurarnos de que el directorio existe
    os.makedirs(os.path.dirname(ruta_elegida), exist_ok=True)
    
    # Guardar el DataFrame CORRECTO (df_clean)
    df_clean.to_csv(ruta_elegida, index=False, encoding='utf-8-sig')
    
    # Verificar que se guardó correctamente
    if os.path.exists(ruta_elegida):
        file_size = os.path.getsize(ruta_elegida) / (1024 * 1024)  # Tamaño en MB
        print(f"✅ ARCHIVO GUARDADO EXITOSAMENTE!")
        print(f"📁 Ruta: {ruta_elegida}")
        print(f"📊 Filas: {df_clean.shape[0]:,}, Columnas: {df_clean.shape[1]}")
        print(f"💾 Tamaño: {file_size:.2f} MB")
        print(f"📅 Guardado el: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Mostrar preview del archivo guardado
        print(f"\n📋 PRIMERAS FILAS DEL ARCHIVO GUARDADO:")
        df_verificacion = pd.read_csv(ruta_elegida)
        print(f"Columnas: {list(df_verificacion.columns)}")
        print(f"Primeras 3 filas:")
        print(df_verificacion.head(3))
        
    else:
        print("❌ Error: El archivo no se pudo guardar")
        
except Exception as e:
    print(f"❌ Error al guardar: {e}")
    print("\n💡 SOLUCIÓN ALTERNATIVA: Guardar en una ruta más simple...")
    
    # Alternativa: guardar en el directorio actual
    ruta_alternativa = "digital_marketing_analysis_cleaned.csv"
    df_clean.to_csv(ruta_alternativa, index=False)
    print(f"✅ Guardado alternativo en: {os.path.abspath(ruta_alternativa)}")

✅ ARCHIVO GUARDADO EXITOSAMENTE!
📁 Ruta: C:\Users\Lucan Vieira\Documents\GitHub\Proyectos\MARKETING\Data\digital_marketing_cleaned.csv
📊 Filas: 2,021, Columnas: 32
💾 Tamaño: 0.20 MB
📅 Guardado el: 2025-10-23 16:27:44

📋 PRIMERAS FILAS DEL ARCHIVO GUARDADO:
Columnas: ['CustomerID', 'Age', 'Income', 'Education_Level', 'Marital_Status', 'Kidhome', 'Teenhome', 'Customer_Days', 'Recency', 'Complain', 'Z_CostContact', 'Z_Revenue', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'MntTotal', 'MntRegularProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmpOverall', 'Response']
Primeras 3 filas:
  CustomerID  Age  Income Education_Level Marital_Status  Kidhome  Teenhome  \
0  CUST_0001   63   58138      Graduation         Single        0         0   
1  CUST_0002   66   46344      Graduation     